In [37]:
pip install pandas transformers torch

In [13]:
import pandas as pd

# Load the dataset
try:
    df = pd.read_csv('/content/spam.csv', encoding='latin1')
    print("Dataset loaded successfully. Here's a preview:")
    print(df.head())
    print("\nDataset info:")
    print(df.info())
except FileNotFoundError:
    print("Error: spam.csv not found. Make sure the file is in the same directory as your script.")
    # Exit or handle the error appropriately if the file isn't found
    exit()

Dataset loaded successfully. Here's a preview:
                                                text target
0  Go until jurong point, crazy.. Available only ...    ham
1                      Ok lar... Joking wif u oni...    ham
2  Free entry in 2 a wkly comp to win FA Cup fina...   spam
3  U dun say so early hor... U c already then say...    ham
4  Nah I don't think he goes to usf, he lives aro...    ham

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5572 non-null   object
 1   target  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


In [14]:
from transformers import pipeline

try:
    classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
    print("\nLLM (text-classification pipeline) loaded successfully.")
    print("Using model:", classifier.model.name_or_path)
except Exception as e:
    print(f"\nError loading LLM: {e}")
    print("Please check your internet connection or try a different model name.")
    exit()


Device set to use cpu



LLM (text-classification pipeline) loaded successfully.
Using model: distilbert-base-uncased-finetuned-sst-2-english


In [16]:
# Function to classify a batch of emails using the loaded LLM
def classify_email_batch(texts):
    predicted_labels = []
    # Loop through each text in the batch
    for text in texts:
        # Prompt Engineering
        prompt = f"Classify the following email as 'ham' or 'spam'. Respond with only 'ham' or 'spam'.\n\nEmail: \"{text}\"\nClassification:"

        try:
            # Step 3b: Perform LLM inference
            llm_output = classifier(prompt)[0]['label']

            # Step 3c: Map LLM output to 'ham' or 'spam' (if necessary)

            if llm_output == 'POSITIVE': # Assuming 'POSITIVE' might relate to ham (or spam depending on context)
                final_label = 'ham'
            elif llm_output == 'NEGATIVE': # Assuming 'NEGATIVE' might relate to spam (or ham)
                final_label = 'spam'
            else:

                final_label = llm_output.lower()

            predicted_labels.append(final_label)

        except Exception as e:
            print(f"Error classifying email: '{text[:50]}...' - {e}")
            predicted_labels.append("error_in_classification")
    return predicted_labels

print("\nBatch classification function defined.")


Batch classification function defined.


In [11]:
# Initialize a list to store all predicted targets
all_predicted_targets = []

# Define batch size (adjust based on your system's memory and LLM's capacity)
batch_size = 1000 # Start with a small batch size, e.g., 1000 or 20000, then increase if stable.

print(f"\nStarting batch processing with batch size: {batch_size}")
print(f"Total emails to process: {len(df)}")

# Process the DataFrame in batches
for i in range(0, len(df), batch_size):
    # Get the current batch of texts
    batch_df = df.iloc[i:i + batch_size]
    batch_texts = batch_df['text'].tolist()

    if not batch_texts: # Skip if batch is empty (can happen at the very end)
        continue

    print(f"Processing emails {i} to {min(i + batch_size, len(df))}...")

    # Get predictions for the current batch using your LLM
    predicted_labels = classify_email_batch(batch_texts)

    # Extend the main list of predictions
    all_predicted_targets.extend(predicted_labels)

print("\nBatch processing complete.")


Starting batch processing with batch size: 1000
Total emails to process: 5572
Processing emails 0 to 1000...
Processing emails 1000 to 2000...
Processing emails 2000 to 3000...
Processing emails 3000 to 4000...
Processing emails 4000 to 5000...
Processing emails 5000 to 5572...

Batch processing complete.


In [17]:
# Add the predicted targets back to the original DataFrame
if len(all_predicted_targets) == len(df):
    df['predicted_target'] = all_predicted_targets
    print("\nPredicted targets added to DataFrame. Here's a sample:")
    print(df[['text', 'target', 'predicted_target']].head(10))

    # Basic Evaluation (Optional, but recommended)
    from sklearn.metrics import accuracy_score, classification_report
    df['target_normalized'] = df['target'].apply(lambda x: x.strip().lower())
    df['predicted_target_normalized'] = df['predicted_target'].apply(lambda x: x.strip().lower())

    # Filter out any 'error_in_classification'
    valid_predictions_df = df[df['predicted_target_normalized'] != 'error_in_classification']

    if not valid_predictions_df.empty:
        true_labels = valid_predictions_df['target_normalized']
        pred_labels = valid_predictions_df['predicted_target_normalized']

        # Get unique labels from both true and predicted sets for consistent reporting
        all_labels = sorted(list(set(true_labels.tolist() + pred_labels.tolist())))

        print(f"\nAccuracy: {accuracy_score(true_labels, pred_labels):.4f}")
        print("\nClassification Report:")
        print(classification_report(true_labels, pred_labels, labels=all_labels, zero_division=0))
    else:
        print("\nNo valid predictions to evaluate.")

else:
    print(f"Mismatch in number of predictions ({len(all_predicted_targets)}) and emails ({len(df)}).")
    print("Something went wrong during batch processing.")

print("\nPractical Exercise 1: Email classification setup complete.")


Predicted targets added to DataFrame. Here's a sample:
                                                text target predicted_target
0  Go until jurong point, crazy.. Available only ...    ham             spam
1                      Ok lar... Joking wif u oni...    ham             spam
2  Free entry in 2 a wkly comp to win FA Cup fina...   spam             spam
3  U dun say so early hor... U c already then say...    ham             spam
4  Nah I don't think he goes to usf, he lives aro...    ham             spam
5  FreeMsg Hey there darling it's been 3 week's n...   spam             spam
6  Even my brother is not like to speak with me. ...    ham             spam
7  As per your request 'Melle Melle (Oru Minnamin...    ham             spam
8  WINNER!! As a valued network customer you have...   spam             spam
9  Had your mobile 11 months or more? U R entitle...   spam             spam

Accuracy: 0.1637

Classification Report:
              precision    recall  f1-score   support

